In [ ]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

In [ ]:
import mlflow
import pandas as pd
datapath = "data/evaluations/"


# Download all evaluation data.

In [ ]:
# Download source data from s3
from gw2ml.data.s3_sync import download_folder_from_s3



download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1763489057', local_folder=f"{datapath}/gw2/")
download_folder_from_s3(s3_folder_prefix='datasources/crypto/raw/1763488125', local_folder=f"{datapath}/crypto/")


In [ ]:
from darts import TimeSeries
from sklearn.preprocessing import MinMaxScaler

def prepare_data_gw2_univariate(input, scaling: bool = True, split_percentage=0.8):

    value_cols = ["buy_unit_price"]
    df = pd.read_csv(input, delimiter=";")
    df.head()
    tsdf = df[value_cols + ['fetched_at']].copy()

        # Convert to datetime and set as index
    tsdf['fetched_at'] = pd.to_datetime(tsdf['fetched_at'])
    # Localize timezone to UTC if it has timezone info, or remove it
    if tsdf['fetched_at'].dt.tz is not None:
        tsdf['fetched_at'] = tsdf['fetched_at'].dt.tz_localize(None)
    tsdf = tsdf.set_index('fetched_at')

    # Resample to exact 5-minute intervals, forward-filling missing values
    tsdf_resampled = tsdf[value_cols].resample('5min').mean().interpolate(method='linear')
    series = TimeSeries.from_dataframe(tsdf_resampled, value_cols=value_cols)
    train, test = series.split(split_percentage)

    if scaling:
        scaler = MinMaxScaler()
        train = scaler.fit_transform(train)
        test = scaler.transform(test)

    return train, test


# def prepare_data_crypto():
#
#     return train, test


In [ ]:
import os

# for file in os.listdir(f"{datapath}/crypto/"):
#     print(file)

gw2_serieses =[]
for file in os.listdir(f"{datapath}/gw2/"):
    print(file)
    train, test = prepare_data_gw2_univariate(f"{datapath}/gw2/{file}")

    gw2_serieses.append({"train": train,
                    "test": test,
                    "item": file})


In [ ]:
gw2_serieses